In [35]:
import json
import math
import pprint
import re
import requests
import sys
import time

from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from operator import itemgetter

YEARS = ["2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
SEASONS = ["2008-09", "2009-10", "2010-11", "2011-12", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19"]
counter = 0

HEADERS = {'user-agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'),
       'referer': 'https://stats.nba.com/players/traditional/'
      }

In [36]:
rookie_data = {
    "metrics": ['per', 'usg', 'bpm', 'mpg', 'winshare', 'ppg', 'rpg', 'apg', 'spg', 'bpg'],
    "stats": [],
    "players": [{'player': 'Kyrie Irving', 'team': '', 'team_short': 'CLE', 'season': '2011-12', 'wins': 0, 'team_games': 0, 'player_games': 51, 'metrics': {'winshare': 4.1,'per': 21.4, 'usg': 28.7, 'bpm': 3.3, 'mpg': 30.5, 'ppg': 18.5, 'rpg': 3.7, 'apg': 5.4, 'spg': 1.1, 'bpg': 0.4}, 'zscores': {'winshare': 0, 'per': 0, 'usg': 0, 'bpm': 0, 'mpg': 0, 'ppg': 0, 'rpg': 0, 'apg': 0, 'spg': 0, 'bpg': 0}, 'total_stand_zscore': 0, 'total_adv_zscore': 0}]
}





In [37]:
def get_standard_deviation(data, key):
    x = 0
    y = 0
    l = len(data)
    for obj in data:
        x = x + obj["metrics"][key]
        y = y + (obj["metrics"][key] ** 2)

    standard_deviation = math.sqrt((y - ((x ** 2) / l)) / (l - 1))
    return standard_deviation

def get_mean(data, key):
    x = 0;
    for obj in data:
        x = x + obj["metrics"][key]

    mean = x / len(data);
    return mean;

def set_stats(data):
    for metric in data["metrics"]:
        standard_dev = get_standard_deviation(data["players"], metric)
        mean = get_mean(data["players"], metric)

        target_metric = {
            "metric": metric,
            "standard_dev": standard_dev,
            "mean": mean,
        }

        data["stats"].append(target_metric)


# ------------------------------------------------------------------------------
# DEFINIING THE INITIAL PLAYER SET

# we define our inital set of players by hitting the NBA's api for rookie stats

def get_player_set(YEAR, counter):

    first_100 = requests.get("https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplay-index%2Fpsl_finder.cgi%3Frequest%3D1%26match%3Dsingle%26type%3Dtotals%26per_minute_base%3D36%26per_poss_base%3D100%26season_start%3D1%26season_end%3D1%26lg_id%3DNBA%26age_min%3D0%26age_max%3D99%26is_playoffs%3DN%26height_min%3D0%26height_max%3D99%26year_min%3D{0}%26year_max%3D{1}%26birth_country_is%3DY%26as_comp%3Dgt%26as_val%3D0%26pos_is_g%3DY%26pos_is_gf%3DY%26pos_is_f%3DY%26pos_is_fg%3DY%26pos_is_fc%3DY%26pos_is_c%3DY%26pos_is_cf%3DY%26qual%3Dpts_per_g_req%26c1stat%3Dper%26c1comp%3Dgt%26c1val%3D-100%26c2stat%3Dws%26c2comp%3Dgt%26c2val%3D-100%26c3stat%3Dbpm%26c3comp%3Dgt%26c3val%3D-100%26c4stat%3Dusg_pct%26c4comp%3Dgt%26order_by%3Dws&div=div_stats".format(YEAR, YEAR))
    first_100_content = BeautifulSoup(first_100.text, "html.parser")
    first_table = first_100_content.find("tbody")
    first_rows = first_table.findAll("tr")
    
    if len(first_rows) > 99:
        second_100 = requests.get("https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplay-index%2Fpsl_finder.cgi%3Frequest%3D1%26match%3Dsingle%26type%3Dtotals%26per_minute_base%3D36%26per_poss_base%3D100%26season_start%3D1%26season_end%3D1%26lg_id%3DNBA%26age_min%3D0%26age_max%3D99%26is_playoffs%3DN%26height_min%3D0%26height_max%3D99%26year_min%3D{0}%26year_max%3D{1}%26birth_country_is%3DY%26as_comp%3Dgt%26as_val%3D0%26pos_is_g%3DY%26pos_is_gf%3DY%26pos_is_f%3DY%26pos_is_fg%3DY%26pos_is_fc%3DY%26pos_is_c%3DY%26pos_is_cf%3DY%26qual%3Dpts_per_g_req%26c1stat%3Dper%26c1comp%3Dgt%26c1val%3D-100%26c2stat%3Dws%26c2comp%3Dgt%26c2val%3D-100%26c3stat%3Dbpm%26c3comp%3Dgt%26c3val%3D-100%26c4stat%3Dusg_pct%26c4comp%3Dgt%26order_by%3Dws&offset%3D100&div=div_stats".format(YEAR, YEAR))
        second_100_content = BeautifulSoup(second_100.text, "html.parser")
        second_table = second_100_content.find("tbody")
        second_rows = second_table.findAll("tr")

        all_br_rows = first_rows + second_rows
    else: 
        all_br_rows = first_rows
        
    for row in all_br_rows:
        try: 
            player_team = row.find("td", {"data-stat": "team_id"}).text
        except AttributeError: 
            continue;
        
        # accounting for special cases where NBA's tricode for a team differs
        # slightly from basketball references'
        if player_team == "PHX":
            team_short = "PHO"
        elif player_team == "CHA" and int(YEAR) >= 2015:
            team_short = "CHO"
        elif player_team =="BKN":
            team_short = "BRK"
        else:
            team_short = player_team
            
        player_name = row.find("td", {"data-stat": "player"}).text
        
        if player_name == "Mohamed Bamba":
            player_name = "Mo Bamba"
            
        games = int(row.find("td", {"data-stat": "g"}).text)
        minutes = int(row.find("td", {"data-stat": "mp"}).text)
        points = int(row.find("td", {"data-stat": "pts"}).text)
        rebounds = int(row.find("td", {"data-stat": "trb"}).text)
        assists = int(row.find("td", {"data-stat": "ast"}).text)
        steals = int(row.find("td", {"data-stat": "stl"}).text)
        blocks = int(row.find("td", {"data-stat": "blk"}).text)
                                
        rookie = {
            "player": player_name,
            "team": "", # this comes from basketball reference standings scrape later
            "team_short": player_team,
            "season": SEASONS[counter],
            "wins": 0,
            "team_games": 0, # this comes from basketball reference later
            "player_games": row.find("td", {"data-stat": "g"}).text,
            # advanced metrics get added via basketball reference later
            "metrics": {
                "winshare": float(row.find("td", {"data-stat": "ws"}).text),
                "per": float(row.find("td", {"data-stat": "per"}).text),
                "usg": float(row.find("td", {"data-stat": "usg_pct"}).text),
                "bpm": float(row.find("td", {"data-stat": "bpm"}).text),
                "mpg": round((minutes / games), 1),
                "ppg": round((points / games), 1), # this comes from nba scrape later
                "rpg": round((rebounds / games), 1), # this comes from nba scrape later
                "apg": round((assists / games), 1), # this comes from nba scrape later
                "spg": round((steals / games), 1), # this comes from nba scrape later
                "bpg": round((blocks / games), 1) # this comes from nba scrape later
            },
            # zscores get calculated later
            "zscores": {
                "winshare": 0,
                "per": 0,
                "usg": 0,
                "bpm": 0,
                'mpg': 0,
                "ppg": 0,
                "rpg": 0,
                "apg": 0,
                "spg": 0,
                "bpg": 0
            }
        }

            
        rookie_data["players"].append(rookie)

            
            
# ------------------------------------------------------------------------------
# GETTING THE NBA STANDINGS SO WE CAN GET TEAM GAMES AND WINS

# get the current nba standings so we can add in team games and team names to the
# player dictionaries later

def get_standings(year):
    # getting the expanded standings broken down into rows
    standings_request = requests.get("https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fleagues%2FNBA_{0}_standings.html&div=div_expanded_standings".format(year))
    standings_content = BeautifulSoup(standings_request.text, "html.parser")
    standings_table = standings_content.find("table", {"id": "expanded_standings"})
    standings_rows = standings_table.find("tbody").findAll("tr")

    # for each row ...
    for row in standings_rows:
        try: 
            # determine the team's record, wins and losses, wins games and team abbreviation
            record = row.find("td", {"data-stat": "Overall"}).text
            wins_losses = record.split("-")
            wins = int(wins_losses[0])
            games = int(wins_losses[0]) + int(wins_losses[1])
            link = row.find("td", {"data-stat": "team_name"}).find("a").get("href")
            split_link = link.split("teams/")
            team_short = split_link[1].split("/")[0]

            # create the team object
            team = {}
            team["team_name"] = row.find("td", {"data-stat": "team_name"}).text
            team["record"] = record
            team["wins"] = wins
            team["games"] = games
            team["short_name"] = team_short

            # append it to the standings list
            standings[year].append(team)
        except AttributeError:
            continue

# ------------------------------------------------------------------------------
# CROSS WALKING PLAYERS WITH TEAMS
# function matches players with long team names and total games played by team

def assign_games(players, standings):
    # iterate over the players
    for player in players:
        # pull that player's tricode from their dictionary
        player_team = player["team_short"]
        # finding the team with the matching tricode in the standings
        try: 
            team = next((team for team in standings if team["short_name"] == player_team))
        except StopIteration:
            print("Alert!!!: ", player_team, player)
        # assign long team name and team games to player
        player["team"] = team["team_name"]
        player["team_games"] = team["games"]
            

def get_traditional_stats(year):

    # the payload needed for the api request
    payload = {
        "College": "",
        "Conference": "",
        "Country": "",
        "DateFrom": "",
        "DateTo": "",
        "Division": "",
        "DraftPick": "",
        "DraftYear": "",
        "GameScope": "",
        "GameSegment": "",
        "Height": "",
        "LastNGames": 0,
        "LeagueID": "00",
        "Location": "",
        "MeasureType": "Base",
        "Month": 0,
        "OpponentTeamID": 0,
        "Outcome": "",
        "PORound": 0,
        "PaceAdjust": "N",
        "PerMode": "PerGame",
        "Period": 0,
        "PlayerExperience": "Rookie",
        "PlayerPosition": "",
        "PlusMinus": "N",
        "Rank": "N",
        "Season": year,
        "SeasonSegment": "",
        "SeasonType": "Regular Season",
        "ShotClockRange": "",
        "StarterBench": "",
        "TeamID": "0",
        "VsConference": "",
        "VsDivision": "",
        "Weight": ""
    }

    # the actual request call
    r = requests.get("http://stats.nba.com/stats/leaguedashplayerstats", params=payload, headers=HEADERS, timeout=120)
    r.raise_for_status()

    # converting the response to json
    nba_player_data = r.json()
    for player in rookie_data["players"]:
        
        hit = False;
        for row in nba_player_data["resultSets"][0]["rowSet"]:
            try:
                if fuzz.token_sort_ratio(player["player"], row[1]) >= 80:
                    player["metrics"]["ppg"] = row[29]
                    player["metrics"]["rpg"] = row[21]
                    player["metrics"]["apg"] = row[22]
                    player["metrics"]["spg"] = row[24]
                    player["metrics"]["bpg"] = row[25]
                    player["metrics"]["mpg"] = row[9]
                    hit = True
            except AttributeError:
                continue
                                
def calculate_z_scores(data):
    for stat in data["stats"]:
        metric = stat["metric"]
        sd = stat["standard_dev"]
        mean = stat["mean"]
        for player in data["players"]:
            player_metric = player["metrics"][metric]
            player["zscores"][metric] = (player_metric - mean) / sd

    for player in data["players"]:
        player["total_stand_zscore"] = player["zscores"]["ppg"] + player["zscores"]["rpg"] + player["zscores"]["apg"] + player["zscores"]["spg"] + player["zscores"]["bpg"]

        player["total_adv_zscore"] = player["zscores"]["mpg"] + player["zscores"]["winshare"] + player["zscores"]["bpm"] + player["zscores"]["per"] + player["zscores"]["usg"]

    data["players"] = sorted(data["players"], key=lambda x: (
        x['total_adv_zscore'], x['total_stand_zscore']))
    data["players"].reverse()


In [38]:
def perform_scrape(counter):
    for YEAR in YEARS: 
        print(YEAR)
        # get the advanced metrics for our remaining players
        get_player_set(YEARS[counter], counter)
    
        # get the initial player set
        # get_traditional_stats(SEASONS[counter])
        
        time.sleep(3)
        counter += 1
    
    # calculate the standard deviation and mean for each metric
    set_stats(rookie_data)

    # calculate the z scores for each player in each metric
    calculate_z_scores(rookie_data)

perform_scrape(counter)

print("Advanced")
print("_________")

for rookie in rookie_data['players'][:50]:
    print("{0}, {1}, {2}".format(rookie["player"], rookie['season'], rookie["total_adv_zscore"]))

rookie_data["players"] = sorted(rookie_data["players"], key=lambda x: (
    x['total_stand_zscore'], x['total_adv_zscore']))
rookie_data["players"].reverse()

print("===========")
print("Traditional")
print("_________")
for rookie in rookie_data['players'][:50]:
    print("{0}, {1}, {2}".format(rookie["player"], rookie['season'], rookie["total_stand_zscore"]))


2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
Advanced
_________
Blake Griffin, 2010-11, 13.466718559271058
Ben Simmons, 2017-18, 11.388576703939748
Karl-Anthony Towns, 2015-16, 11.162729412987868
Luka Doncic, 2018-19, 10.174233987141434
Kyrie Irving, 2011-12, 9.513441032574065
Tyreke Evans, 2009-10, 8.960400908699093
Donovan Mitchell, 2017-18, 8.281061130318367
Nikola Jokic, 2015-16, 8.151323239665984
Damian Lillard, 2012-13, 7.807630526117558
Deandre Ayton, 2018-19, 7.262367682434526
Stephen Curry, 2009-10, 6.576359304223678
Brandon Jennings, 2009-10, 6.194567285749216
Derrick Rose, 2008-09, 6.1801688065825635
Jayson Tatum, 2017-18, 6.121750054327318
Marc Gasol, 2008-09, 5.997057134511426
Greg Monroe, 2010-11, 5.934598733479545
Brook Lopez, 2008-09, 5.920250740929112
Kristaps Porzingis, 2015-16, 5.897211942078502
Mitchell Robinson, 2018-19, 5.712547952385986
Rudy Fernandez, 2008-09, 5.582165938184897
Nikola Mirotic, 2014-15, 5.390577500923449
O.J. Mayo, 2008-09, 5.3341956987

In [39]:
past_rookies = []

for rookie in rookie_data['players']:
    this_rookie = {
        "name": rookie["player"],
        "team": rookie["team_short"],
        "year": rookie["season"],
        "traditional": rookie["total_stand_zscore"],
        "advanced": rookie["total_adv_zscore"],
    }
    
    if this_rookie["name"] == "Blake Griffin": 
        print(rookie)
    past_rookies.append(this_rookie)
    
print(len(past_rookies))

{'player': 'Blake Griffin', 'team': '', 'team_short': 'LAC', 'season': '2010-11', 'wins': 0, 'team_games': 0, 'player_games': '82', 'metrics': {'winshare': 9.8, 'per': 21.9, 'usg': 27.3, 'bpm': 3.2, 'mpg': 38.0, 'ppg': 22.5, 'rpg': 12.1, 'apg': 3.8, 'spg': 0.8, 'bpg': 0.5}, 'zscores': {'winshare': 3.9932796523972995, 'per': 2.7070221618201717, 'usg': 2.0935624376831203, 'bpm': 2.203167636519901, 'mpg': 2.4696866708505656, 'ppg': 3.3647015576434622, 'rpg': 4.378534379987115, 'apg': 1.21286994200271, 'spg': 0.26219279508478815, 'bpg': 0.19941998510756154}, 'total_stand_zscore': 9.417718659825638, 'total_adv_zscore': 13.466718559271058}
228


In [40]:
with open("../data/all_past_rookies.json", "w") as write_file:
    json.dump(past_rookies, write_file)